In [9]:
import os 
import pandas as pd
import path
import pickle 
import numpy as np

In [10]:
base_path_junctions = '/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes'

In [17]:
folder_OV_jx_unfiltered = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv'
folder_OV_jx_conf2 = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv'

folfer_BRCA_jx_unfiltered = ''

count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.rechunked.hdf5')
splice_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_OV = '/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_Ov_378.tsv'

count_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_BRCA = "/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_BRCA_1102.tsv"

In [34]:
graph, meta = pickle.load(open(splice_path_OV, 'rb'))

In [64]:
sample = 'sample1' # relink samples 1 to 5

In [65]:
path_OV_jx_unfiltered = os.path.join(base_path_junctions, folder_OV_jx_unfiltered, sample)
print(path_OV_jx_unfiltered)
path_OV_jx_conf2 = os.path.join(base_path_junctions, folder_OV_jx_conf2, sample)
print(path_OV_jx_conf2)
path_OV_jx_unfiltered = os.path.realpath(path_OV_jx_unfiltered)
print(path_OV_jx_unfiltered)
path_OV_jx_conf2 = os.path.realpath(path_OV_jx_conf2)
print(path_OV_jx_conf2)

df_OV_jx_unfiltered = pd.read_csv(path_OV_jx_unfiltered,
                                  sep = '\t')
df_OV_jx_conf2 = pd.read_csv(path_OV_jx_conf2, 
                                  sep = '\t')

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/TCGA-25-1319-01A-01R-1565-13.all.junctions.tsv.gz
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/TCGA-25-1319-01A-01R-1565-13.all.conf_2.filt.junctions.tsv.gz


In [66]:
def unique_junctions_df(df):
    display(df.head(3))
    print('shape table')
    print(df.shape)
    print('shape table excluding column count')
    print(df.drop(['count'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, chr')
    print(df.drop(['count', 'chr'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand')
    print(df.drop(['count', 'strand'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand, chr')
    print(df.drop(['count', 'strand', 'chr'], axis = 1).drop_duplicates().shape)
    

In [67]:
print(path_OV_jx_unfiltered)
unique_junctions_df(df_OV_jx_unfiltered)
print(path_OV_jx_conf2)
unique_junctions_df(df_OV_jx_conf2)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/TCGA-25-1319-01A-01R-1565-13.all.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,3
2,chr10,-,277577,281199,12


shape table
(485934, 5)
shape table excluding column count
(485934, 4)
shape table excluding column count, chr
(485747, 3)
shape table excluding column count, strand
(485436, 3)
shape table excluding column count, strand, chr
(485249, 2)
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/TCGA-25-1319-01A-01R-1565-13.all.conf_2.filt.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,1
2,chr10,-,277577,281199,9


shape table
(396650, 5)
shape table excluding column count
(396650, 4)
shape table excluding column count, chr
(396510, 3)
shape table excluding column count, strand
(396405, 3)
shape table excluding column count, strand, chr
(396265, 2)


# Graph

In [86]:
junction_all_genes = []
for idx, gene in enumerate(graph):
    if idx % 1000 == 0:
        print(idx)
    gene.from_sparse()
    edges_coords = np.array([(a,b) for a, b in zip(np.where(gene.splicegraph.edges)[0],np.where(gene.splicegraph.edges)[1])])
    junction_gene = [(gene.splicegraph.vertices[0][i], gene.splicegraph.vertices[1][j]) for i, j in edges_coords]
    junction_all_genes.extend(junction_gene)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000


In [101]:
df_junction_gene = pd.DataFrame(junction_all_genes, columns = ['junction_start', 'junction_end'])


In [102]:
df_junction_gene.head()

,junction_start,junction_end
0,11011,12057
1,11471,12057
2,11644,12057
3,11868,12696
4,11868,12697


In [103]:
df_junction_gene.shape

(21054558, 2)

In [104]:
df_junction_gene.drop_duplicates().shape

(7270711, 2)

In [105]:
jxS_graph_unfilt = df_junction_gene.drop_duplicates().merge(\
    df_OV_jx_unfiltered.loc[:, ['junction_start', 'junction_end']].drop_duplicates(), 
                                          on = ['junction_start', 'junction_end'], how = 'inner')

In [106]:
jxS_graph_unfilt.drop_duplicates().shape

(561, 2)

# Graph Single Gene

In [37]:
gene = graph[1]
gene.from_sparse()

In [38]:
#vars(gene.splicegraph)

{'vertices': array([[14403, 14403, 14403, 14403, 14403, 14403, 14403, 14403, 14929,
         14969, 15004, 15004, 15020, 15339, 15795, 15795, 15795, 15795,
         15795, 15795, 15795, 15795, 15827, 15827, 15827, 15870, 15915,
         15916, 16089, 16110, 16606, 16606, 16606, 16606, 16606, 16606,
         16606, 16637, 16710, 16722, 16723, 16733, 16738, 16853, 16857,
         16857, 16857, 16875, 16965, 17029, 17232, 17232, 17605, 17605,
         17605, 17699, 17914, 18267, 18267, 18267, 18267, 18267, 18267,
         18267, 18354, 18500, 18500, 18500, 18500, 18500, 18500, 18875,
         18912, 18912, 18912, 18912, 18912, 18912, 18939, 18953, 19124,
         20086, 22427, 23272, 23709, 23730, 23730, 23730, 24217, 24266,
         24737, 24737, 25217, 27195, 28564, 29320, 29533, 29539, 29602,
         31592],
        [14501, 14548, 14593, 14664, 14690, 14787, 14829, 14843, 15038,
         15038, 15038, 15059, 15038, 15539, 15837, 15859, 15942, 15947,
         16027, 16289, 16310, 16765

In [39]:
#[(a,b) for a, b in zip(np.where(gene.segmentgraph.seg_edges)[0],np.where(gene.segmentgraph.seg_edges)[1])]

[(0, 9),
 (0, 10),
 (1, 10),
 (1, 43),
 (2, 9),
 (2, 10),
 (2, 14),
 (3, 10),
 (3, 19),
 (3, 20),
 (4, 10),
 (4, 29),
 (5, 10),
 (5, 44),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (6, 14),
 (6, 18),
 (6, 29),
 (7, 9),
 (7, 10),
 (7, 14),
 (11, 14),
 (11, 18),
 (11, 29),
 (12, 14),
 (13, 14),
 (15, 31),
 (16, 29),
 (20, 29),
 (21, 29),
 (22, 29),
 (22, 43),
 (23, 29),
 (26, 29),
 (27, 29),
 (27, 43),
 (29, 34),
 (31, 37),
 (32, 38),
 (32, 40),
 (34, 43),
 (35, 39),
 (40, 42),
 (40, 43),
 (40, 44),
 (41, 43),
 (41, 44),
 (44, 47),
 (45, 48),
 (48, 49),
 (49, 51),
 (51, 53),
 (53, 55),
 (54, 55),
 (54, 91),
 (55, 56),
 (55, 64),
 (55, 69),
 (55, 83),
 (55, 91),
 (55, 96),
 (55, 97),
 (55, 98),
 (56, 69),
 (56, 91),
 (58, 91),
 (59, 69),
 (59, 91),
 (60, 64),
 (60, 69),
 (60, 80),
 (60, 83),
 (60, 91),
 (60, 96),
 (60, 97),
 (60, 100),
 (61, 64),
 (61, 69),
 (61, 80),
 (61, 83),
 (61, 91),
 (61, 96),
 (61, 97),
 (61, 100),
 (62, 64),
 (62, 69),
 (62, 83),
 (62, 91),
 (62, 96),
 (62, 97),
 (64

In [43]:
edges_coords = np.array([(a,b) for a, b in zip(np.where(gene.splicegraph.edges)[0],np.where(gene.splicegraph.edges)[1])])

In [51]:
gene.splicegraph.vertices.shape

(2, 100)

In [58]:
# CHECK ORDER
junction_gene = [(gene.splicegraph.vertices[0][i], gene.splicegraph.vertices[1][j]) for i, j in edges_coords]

In [107]:
junction_gene[0:10]

[(57212183, 57213357),
 (57212183, 57213964),
 (57212183, 57213357),
 (57212183, 57213602),
 (57212975, 57213964),
 (57213203, 57213095),
 (57213203, 57213125),
 (57213203, 57213602),
 (57213525, 57213357),
 (57213525, 57213357)]

In [63]:
df_junction_gene = pd.DataFrame(junction_gene, columns = ['junction_start', 'junction_end'])

In [75]:
junction_graph_unfilt = df_junction_gene.merge(df_OV_jx_unfiltered, on = ['junction_start', 'junction_end'], how = 'left')

In [80]:
display(junction_graph_unfilt.head())
print(junction_graph_unfilt.shape)
print(junction_graph_unfilt.drop_duplicates().shape)
junction_graph_unfilt = junction_graph_unfilt.merge(df_OV_jx_unfiltered, on = ['junction_start', 'junction_end'], how = 'left')

,junction_start,junction_end,chr,strand,count
0,14403,15038,NaN,NaN,NaN
1,14403,15038,NaN,NaN,NaN
2,14403,15059,NaN,NaN,NaN
3,14403,15038,NaN,NaN,NaN
4,14403,17055,NaN,NaN,NaN


(478, 5)
(254, 5)


In [81]:
junction_graph_unfilt.head()

,junction_start,junction_end,chr_x,strand_x,count_x,chr_y,strand_y,count_y
0,14403,15038,NaN,NaN,NaN,NaN,NaN,NaN
1,14403,15038,NaN,NaN,NaN,NaN,NaN,NaN
2,14403,15059,NaN,NaN,NaN,NaN,NaN,NaN
3,14403,15038,NaN,NaN,NaN,NaN,NaN,NaN
4,14403,17055,NaN,NaN,NaN,NaN,NaN,NaN
